### SMI OLOG

SMI Olog (https://logbook.nsls2.bnl.gov/12-ID/)

In [2]:
%matplotlib notebook
from pySMI.smi_libs import *
from pySMI.smi_package import *
plt.rcParams.update({'figure.max_open_warning': 0})
plt.rcParams.update({ 'image.origin': 'lower'   })
plt.rcParams.update({ 'image.interpolation': 'none'   })



In [ ]:
%run ~/pySMI/pySMI/smi_generic_functions.py

In [ ]:
beam_x = 98.0 #97.0 for 2.4 keV; 98 for 16 keV
beam_y = 1316.0   #1475 - 159

energy = 16.0 #2.484 #keV

start_angle = 4.0 #in degree, we will change according to data 
phi_spacing= 6.0  #previously 4 deg



In [ ]:
wavelength_A =12.39842/energy
sam_det = 0.275 #in meters
dq = 4*np.pi/wavelength_A  * (172/1e6/sam_det)

calibration = CalibrationGonio(wavelength_A= wavelength_A) # 20.0 keV
calibration.set_image_size(195, height=1475) # Pilatus300kW vertical
#calibration.set_image_size(1475, height=195) # Pilatus300kW horizontal
calibration.set_pixel_size(pixel_size_um=172.0)
calibration.set_beam_position(beam_x,  beam_y )
calibration.set_distance( sam_det )
calibration.set_angles(det_phi_g=0., det_theta_g=0., )
print('ratio Dw = {:.3f}'.format(calibration.get_ratioDw()))
mask_dir = '/GPFS/xf12id1/analysis/commissioning/masks/'
mask = Mask(mask_dir+'Pilatus300kWh_main_gaps-mask.png')
#mask.load('./Pilatus300kWh_current-mask.png')
mask.load(mask_dir+'Pilatus300kWh_current-mask.png')

# Define Cycle &proposal_id here

In [ ]:
CYCLE='2017_3'
proposal_id =  '302771'




In [ ]:
inDir = '/GPFS/xf12id1/data/300KW/images/users/%s/'%proposal_id

In [ ]:
fp = get_base_all_filenames(inDir,base_filename_cut_length=-12-3 )
print('We found %s samples in %s folder.' %(len( list( fp.keys() ) ), inDir ) )

# Create ResDir Here

In [ ]:
ResDir0 = os.path.join('/GPFS/xf12id1/analysis/2017_3/', proposal_id, 'Results/Ramya/')
os.makedirs(ResDir0, exist_ok=True)
print('Results from this analysis will be stashed in the directory %s' % ResDir0)

In [ ]:
for k in list(fp.keys()):
    print(k)

In [ ]:

#%run ~/pySMI/pySMI/Stitching.py

# Select a interested data set here

In [ ]:

k = 'BSpec_RT_FullSample_ExoMeso_Coarse_Scan_'

In [ ]:
k

In [ ]:
ResDir = os.path.join( ResDir0, k +'/' )
os.makedirs(ResDir, exist_ok=True)
print('Results from this analysis will be stashed in the directory %s' % ResDir)


In [ ]:
infiles = sorted( fp[k] )
print(len(infiles))

# Check raw data

In [ ]:
ResDir

In [ ]:
vmin= 10
vmax= 7e2

In [ ]:
Ns = 0
Ni = 5

In [ ]:


fig = plt.figure()# figsize=[2,8]) 
for i in range(Ns, Ns+Ni):
    #print(i)
    ax = fig.add_subplot(1, Ni+1, i+1)
    #d = process.load(  infiles[i]  )
    
    d = np.array(  PIL.Image.open(infiles[i]).convert('I') )
    #vmin=  max(1, d.mean() - d.std()*sca), vmax= d.mean() + d.std()*sca,
    sca= 1
    show_img( (d).T, logs = True, show_colorbar= False,show_ticks =False,
             ax= [fig, ax], image_name= '%02d'%(i+1), cmap = cmap_vge_hdr, 
             vmin= vmin, vmax= vmax,              
            aspect=1, save=False, path=ResDir)
fig.savefig(ResDir + k+'raw_image_%s-%s.png'%(Ns, Ns+Ni))

# Set data for stitching

In [ ]:
Ns = 205 + 6  #Ns = 210 is a good for benchmark
Ns = 90  
Ni = 5

if Ns%2:
    snake = True
    print('This is a snake scan.')
else:
    snake = False
if snake:
    infiles = sorted( fp[k] )[ Ns: Ns+Ni][::-1]
else:
    infiles = sorted( fp[k] )[ Ns: Ns+Ni] 

print( sorted(fp[k])[Ns])
#phis = np.array( [get_phi(infile, phi_offset= 0, phi_start=4.5, phi_spacing= 4.0, ext='_WAXS.tif') for infile in infiles]     )
start_angle = 5.0
phi_spacing = 6  
phis = -start_angle - np.arange(0, Ni) * phi_spacing 
print('The rotated angles are: %s.'%phis)


In [ ]:
overlap_width = get_pixel_overlap( phi_spacing )
print( overlap_width )

# Find the scaling factor for stitiching
* define the smoothing parameter (w, p)  in a savgol_filter algorithm

In [ ]:
w= 801

if w>=101:
    p = 5
else:
    p=3
    
print(w,p)    
    

# Stich by giving filenames

In [ ]:
infiles

In [ ]:
data, dataM, scale,scale_smooth = Correct_Overlap_Images_Intensities( infiles, window_length=w, polyorder=p, 
                            overlap_width= overlap_width, badpixel_width = 10  )

check_overlap_scaling_factor( scale,scale_smooth, i=1, 
                             filename = ResDir + k + 'smoothed_one_line_1_%s-%s.png'%(Ns, Ns+Ni), 
                             save= True )


# OR Stitch by giving data

In [ ]:
fps = [  'BSpec_RT_FullSample_ExoMeso_Coarse_Scan_%03d_WAXS.tif'%i for i in range(91, 96)]
print(fps)

In [ ]:
Data = np.zeros( 5, dtype= object )
for j, fp in enumerate( fps ):
    Data[j] = np.array(  PIL.Image.open( inDir + fp    ).convert('I') )


In [ ]:
data, dataM, scale,scale_smooth = Correct_Overlap_Images_Intensities( infiles, Data=Data,
                                                window_length=w, polyorder=p, 
                            overlap_width= overlap_width, badpixel_width = 10  )

check_overlap_scaling_factor( scale,scale_smooth, i=1, 
                             filename = ResDir + k + 'smoothed_one_line_1_%s-%s.png'%(Ns, Ns+Ni), 
                             save= True )


# Stitching data here

In [ ]:
angzs, angze, angxs, angxe = [
    np.degrees( np.arctan2 ( (beam_y-data.shape[0]) * 172/1e6, sam_det ) ), 
    np.degrees( np.arctan2 ( ( beam_y ) * 172/1e6, sam_det ) ),
    abs(phis[0]) - 3.5, 
    abs(phis[ len(phis)-1]) + 3.5     ] 

ytick = np.round( np.linspace(angzs, angze, 9 ), 0 )
xtick = np.round( np.linspace(angxs, angxe, 8 ), 0 )
axs =  np.linspace(angxs, angxe, data.shape[1] )
azs =  np.linspace(angzs, angze, data.shape[0] )

In [ ]:
print( angxs, angxe, angzs, angze )

# Stitching data in angle space

In [ ]:


fig,ax=plt.subplots()
show_img(data, logs = False,  ax=[fig,ax], 
         vmin=vmin, vmax=vmax,
         cmap = cmap_vge_hdr,  extent=[angxs, angxe, angzs, angze], aspect=1,  
         image_name = k+ 'Angle_%s-%s'%(Ns, Ns+Ni), dpi=500, save=False, path=ResDir)
ax.set_yticks( ytick )
ax.set_xticks( xtick )

fig.savefig( ResDir +  k+ 'Angle_%s-%s.png'%(Ns, Ns+Ni) )

# Stitching data in q-space

In [ ]:
qdata, xs, zs = stitch_WAXS_in_Qspace( dataM, phis, calibration, x='Qx',
                              dx= 0, dy = 22, dz = 0,  dq= dq, mask=mask.data.T )
    
show_img(qdata, logs = False,  vmin=  vmin, vmax= vmax,
         extent=[xs[0], xs[-1], zs[0],zs[-1]], xlabel='$q_x \, (\AA^{-1})$',  ylabel='$q_z \, (\AA^{-1})$', 
         cmap = cmap_vge_hdr,   aspect=1,  image_name = k + 'Q_pos_%s-%s'%(Ns, Ns+Ni), 
         save=True, path=ResDir)



# Show the stitched image with a ring

In [ ]:
if False:
    bx,by = np.argmin( np.abs(xs) ), np.argmin( np.abs(zs) )

    ring_mask = create_ring_mask( qdata.shape, 103, 184, [bx,by]  )

    show_img(qdata, label_array=ring_mask, logs = False, 
         vmin=  max(0,qdata.mean() -qdata.std()*sca), vmax= qdata.mean() + qdata.std()*sca,          
         cmap = cmap_vge_hdr,   aspect=1,  image_name = k + 'Q')


# Save data

xx, zz = np.meshgrid( axs, azs)            
m = np.round( np.vstack( [xx.ravel(), zz.ravel(), data.ravel()] ).T, 5) 
np.savetxt( ResDir  + k + 'Angle_data_%s_%s.txt'%(Ns, Ns+5),   m,fmt='%.5e', ) 
#np.save(  ResDir  + k + 'Angle_data', data )

xx, zz = np.meshgrid( xs, zs)            
m = np.round( np.vstack( [xx.ravel(), zz.ravel(), qdata.ravel()] ).T, 5) 
np.savetxt( ResDir  + k + 'Q_data_%s_%s.txt'%(Ns, Ns+5),   m,fmt='%.5e', ) 
#np.save(  ResDir  + k + 'Angle_data', data )

In [ ]:
save_array_to_tiff(  qdata, ResDir + k + 'Qmap_%s-%s.tif'%(Ns, Ns+Ni)  )
np.savetxt(  ResDir + k + 'Qmap_qx_cor_%s-%s.txt'%(Ns, Ns+Ni),  xs )
np.savetxt(  ResDir + k + 'Qmap_qz_cor_%s-%s.txt'%(Ns, Ns+Ni),  zs )